# Libraries and Google Drive

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

from owslib.wms import WebMapService
import matplotlib.pyplot as plt

from PIL import Image
from IPython.display import display
from IPython.display import Image as displayImage

import io
from io import BytesIO
import time
import random
import os

import rasterio
from rasterio.transform import from_bounds
from rasterio.plot import show
from rasterio.crs import CRS
import rasterio.features
from rasterio.features import geometry_mask

import shapely
from shapely.geometry import Polygon, LineString

from pyproj import Geod
from pyproj import Transformer
from geopy.distance import distance

import math

import ast

/var/folders/87/npkg34b55xs9w2wkx4nfzg5h0000gn/T/ipykernel_11853/1887574735.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [107]:
# Mount Drive
#from google.colab import drive
#drive.mount("/content/drive")

In [2]:
# Set wd
os.chdir('/Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas')

print("Working directory:", os.getcwd())

Working directory: /Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas


# Import csv of correctly labelled service stations and extract associated polygons

In [4]:
# Load verified_service_stations csv and transform to list
# verified_service_stations_path = "02_data_acquisition/verified_parking_data/verified_service_stations.csv"
verified_service_stations_path = "02_data_acquisition/verified_parking_data/verified_service_stations_incl_quick.csv"

# Read the CSV file into a pandas DataFrame
verified_service_stations = pd.read_csv(verified_service_stations_path)

# Transform df to list
verified_service_stations_list = []
for row in verified_service_stations.values.tolist():
    verified_service_stations_list.extend(row)

# Print first 5 items
print(verified_service_stations[:5])
print("\n")
print("Number of correctly labelled service stations:", len(verified_service_stations_list))

                         id_rest
0   lon_8.6630029_lat_50.2542348
1  lon_11.3531491_lat_50.9378362
2   lon_10.2712414_lat_53.331074
3   lon_9.6300119_lat_53.8303266
4   lon_9.6297338_lat_53.8301666


Number of correctly labelled service stations: 296


In [5]:
# Load the GeoJSON files containing service station polygons and truck/car parking polygons together with service station IDs (created in 01a_OSM_rest_stops.ipynb)
#all_parkings = gpd.read_file("02_data_acquisition/parking_data/all_parkings.geojson") # polygons of car and truck parking space
#rest_stations = gpd.read_file("02_data_acquisition/parking_data/rest_stations.geojson") # polygons and ids of rest stations

all_parkings = gpd.read_file("02_data_acquisition/verified_parking_data/parking_corrected_incl_quick.geojson") # polygons of car and truck parking space
rest_stations = gpd.read_file("02_data_acquisition/parking_data/rest_stations.geojson") # polygons and ids of rest stations

In [7]:
rest_stations.shape

(506, 5)

In [8]:
# Create geopandas geodataframes that only contain polygons of correctly labelled service stations

print("Shape of all_parkings before dropping irrelevant rows:", all_parkings.shape)
print("Shape of rest_stations before dropping irrelevant rows:", rest_stations.shape)

# Drop all rows not corresponding to correctly labelled service stations
parking_areas_ver = all_parkings[all_parkings["id_rest"].isin(verified_service_stations_list)]
rest_stations_ver = rest_stations[rest_stations["id"].isin(verified_service_stations_list)]

# Drop all rows corresponding to correctly labelled service stations
parking_areas_wrong = all_parkings[~all_parkings["id_rest"].isin(verified_service_stations_list)]
rest_stations_wrong = rest_stations[~rest_stations["id"].isin(verified_service_stations_list)]

print("Shape of parking_areas_ver after dropping irrelevant rows:", parking_areas_ver.shape)
print("Shape of rest_stations_ver after dropping irrelevant rows:", rest_stations_ver.shape)

print("Shape of parking_areas_wrong after dropping irrelevant rows:", parking_areas_wrong.shape)
print("Shape of rest_stations_wrong after dropping irrelevant rows:", rest_stations_wrong.shape)

Shape of all_parkings before dropping irrelevant rows: (1606, 6)
Shape of rest_stations before dropping irrelevant rows: (506, 5)
Shape of parking_areas_ver after dropping irrelevant rows: (1304, 6)
Shape of rest_stations_ver after dropping irrelevant rows: (296, 5)
Shape of parking_areas_wrong after dropping irrelevant rows: (302, 6)
Shape of rest_stations_wrong after dropping irrelevant rows: (210, 5)


In [9]:
# Drop irrelevant columns, reorder columns, reset index of parkings_areas_ver

parking_areas_ver = parking_areas_ver.drop(["id_car", "name"], axis = 1)
parking_areas_ver = parking_areas_ver.rename(columns={"@id": "id_OSM_parking"})
parking_areas_ver = parking_areas_ver.reindex(columns=["id_rest", "id_OSM_parking", "type", "geometry"])
parking_areas_ver = parking_areas_ver.reset_index(drop = True)

parking_areas_ver.head()

,id_rest,id_OSM_parking,type,geometry
0,lon_10.2712414_lat_53.331074,relation/13277623,car,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53..."
1,lon_7.5846163_lat_53.2588511,way/33526884,car,"POLYGON ((7.58527 53.25883, 7.58618 53.25869, ..."
2,lon_7.2005154_lat_52.2816225,way/38125314,car,"POLYGON ((7.19685 52.28128, 7.19692 52.28127, ..."
3,lon_10.7306355_lat_50.559048,way/48905621,car,"POLYGON ((10.73083 50.55915, 10.73087 50.55881..."
4,lon_10.7298729_lat_50.5605634,way/48925325,car,"POLYGON ((10.72986 50.56033, 10.72980 50.56026..."


In [10]:
# Drop irrelevant columns, reorder columns, reset index of rest_stations_ver

rest_stations_ver = rest_stations_ver.rename(columns={"id": "id_rest", "@id": "id_OSM_rest"})
rest_stations_ver = rest_stations_ver.drop("highway", axis = 1)
rest_stations_ver = rest_stations_ver.reset_index(drop = True)

rest_stations_ver.head()

,id_rest,id_OSM_rest,name,geometry
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"POLYGON ((11.35315 50.93784, 11.35315 50.93783..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"POLYGON ((10.27124 53.33107, 10.27096 53.33121..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"POLYGON ((9.63001 53.83033, 9.63273 53.82905, ..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"POLYGON ((9.62973 53.83017, 9.62965 53.83010, ..."


In [11]:
# Drop irrelevant columns, reorder columns, reset index of parkings_areas_wrong

parking_areas_wrong = parking_areas_wrong.drop(["id_car", "name"], axis = 1)
parking_areas_wrong = parking_areas_wrong.rename(columns={"@id": "id_OSM_parking"})
parking_areas_wrong = parking_areas_wrong.reindex(columns=["id_rest", "id_OSM_parking", "type", "geometry"])
parking_areas_wrong = parking_areas_wrong.reset_index(drop = True)

parking_areas_wrong.head()

,id_rest,id_OSM_parking,type,geometry
0,lon_10.2703215_lat_53.3301407,relation/13277622,car,"MULTIPOLYGON (((10.26903 53.33039, 10.26926 53..."
1,lon_12.140788_lat_52.2431929,relation/15482166,truck,"MULTIPOLYGON (((12.14216 52.24271, 12.14213 52..."
2,lon_12.149456_lat_52.2449316,relation/15482306,car,"MULTIPOLYGON (((12.14713 52.24497, 12.14717 52..."
3,lon_9.323161_lat_54.0686721,way/31129920,car,"POLYGON ((9.32127 54.06898, 9.32314 54.06861, ..."
4,lon_11.2481831_lat_52.1991827,way/42750034,car,"POLYGON ((11.24512 52.19985, 11.24526 52.19971..."


In [12]:
# Drop irrelevant columns, reorder columns, reset index of rest_stations_wrong

rest_stations_wrong = rest_stations_wrong.rename(columns={"id": "id_rest", "@id": "id_OSM_rest"})
rest_stations_wrong = rest_stations_wrong.drop("highway", axis = 1)
rest_stations_wrong = rest_stations_wrong.reset_index(drop = True)

rest_stations_wrong.head()

,id_rest,id_OSM_rest,name,geometry
0,lon_11.2442867_lat_51.3248863,relation/3947321,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,Probstheide,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,relation/4672228,Ladeburger Heide,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6624068_lat_50.2544468,way/22568868,Spießwald,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."
4,lon_8.6184146_lat_49.7896285,way/28329065,Rastplatz Rolandshöhe,"POLYGON ((8.61841 49.78963, 8.61857 49.79067, ..."


In [13]:
# Check shapes of dataframes

print("Shape of parking_areas_ver:", parking_areas_ver.shape)
print("Shape of rest_stations_ver:", rest_stations_ver.shape)
print("Shape of parking_areas_wrong:", parking_areas_wrong.shape)
print("Shape of rest_stations_wrong:", rest_stations_wrong.shape)

Shape of parking_areas_ver: (1304, 4)
Shape of rest_stations_ver: (296, 4)
Shape of parking_areas_wrong: (302, 4)
Shape of rest_stations_wrong: (210, 4)


In [129]:
# Download geopandas geodataframes as GeoJSONs

# Specify the path where you want to save the GeoJSON file
#output_path_parking_areas_ver = "02_data_acquisition/verified_parking_data/parking_areas_ver_incl_quick.geojson"
#output_path_rest_stations_ver = "02_data_acquisition/verified_parking_data/rest_stations_ver_incl_quick.geojson"
#output_path_parking_areas_wrong = "02_data_acquisition/verified_parking_data/parking_areas_wrong_incl_quick.geojson"
#output_path_rest_stations_wrong = "02_data_acquisition/verified_parking_data/rest_stations_wrong_incl_quick.geojson"

# Export the GeoPandas DataFrame to GeoJSON
#parking_areas_ver.to_file(output_path_parking_areas_ver, driver='GeoJSON')
#rest_stations_ver.to_file(output_path_rest_stations_ver, driver='GeoJSON')
#parking_areas_wrong.to_file(output_path_parking_areas_wrong, driver='GeoJSON')
#rest_stations_wrong.to_file(output_path_rest_stations_wrong, driver='GeoJSON')

# Load GeoJSONs and define bounding boxes for download

In [14]:
# Load the GeoJSON files of verified rest stops
parking_areas_ver = gpd.read_file("02_data_acquisition/verified_parking_data/parking_areas_ver_incl_quick.geojson") # polygons of verified car and truck parking space
rest_stations_ver = gpd.read_file("02_data_acquisition/verified_parking_data/rest_stations_ver_incl_quick.geojson") # polygons of verified rest stops

In [15]:
parking_areas_ver

,id_rest,id_OSM_parking,type,geometry
0,lon_10.2712414_lat_53.331074,relation/13277623,car,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53..."
1,lon_7.5846163_lat_53.2588511,way/33526884,car,"POLYGON ((7.58527 53.25883, 7.58618 53.25869, ..."
2,lon_7.2005154_lat_52.2816225,way/38125314,car,"POLYGON ((7.19685 52.28128, 7.19692 52.28127, ..."
3,lon_10.7306355_lat_50.559048,way/48905621,car,"POLYGON ((10.73083 50.55915, 10.73087 50.55881..."
4,lon_10.7298729_lat_50.5605634,way/48925325,car,"POLYGON ((10.72986 50.56033, 10.72980 50.56026..."
...,...,...,...,...
1299,lon_11.0467665_lat_51.4688222,way/1194289084,truck,"POLYGON ((11.04380 51.46788, 11.04414 51.46784..."
1300,lon_11.0467665_lat_51.4688222,way/1194289085,truck,"POLYGON ((11.04439 51.46815, 11.04445 51.46815..."
1301,lon_11.0467665_lat_51.4688222,way/1194289086,truck,"POLYGON ((11.04503 51.46846, 11.04506 51.46846..."
1302,lon_11.0467665_lat_51.4688222,way/1194289087,quick,"POLYGON ((11.04375 51.46758, 11.04399 51.46763..."


In [16]:
rest_stations_ver

,id_rest,id_OSM_rest,name,geometry
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"POLYGON ((11.35315 50.93784, 11.35315 50.93783..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"POLYGON ((10.27124 53.33107, 10.27096 53.33121..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"POLYGON ((9.63001 53.83033, 9.63273 53.82905, ..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"POLYGON ((9.62973 53.83017, 9.62965 53.83010, ..."
...,...,...,...,...
291,lon_12.7580653_lat_52.8778128,way/915630046,Am Rhinluch,"POLYGON ((12.75807 52.87781, 12.75786 52.87778..."
292,lon_7.2005154_lat_52.2816225,way/988014767,Gut Friedrichstal,"POLYGON ((7.20052 52.28162, 7.20053 52.28167, ..."
293,lon_8.5597202_lat_49.5969765,way/992004892,Wildbahn,"POLYGON ((8.55972 49.59698, 8.55958 49.59693, ..."
294,lon_10.7306355_lat_50.559048,way/1171916133,Adlersberg,"POLYGON ((10.73064 50.55905, 10.73067 50.55851..."


### Find minimum needed image size (pixels) to show maximum resolution

In [17]:
def find_biggest_rest_station(geodataframe):
    """
    Returns the biggest rest station.

    Parameters:
    geodataframe (geopandas geodataframe): contains polygon/multipolygon geometries

    Returns:
    The id of the biggest rest station.
    """

    def polygon_to_bbox(polygon):
        """
        Transforms a polygon into a square bounding box.

        Parameters:
        polygon (Polygon): input polygon

        Returns:
        The resulting square bounding box bounds as a list: [left, bottom, right, top].
        """


        # Calculate the bounding box of the polygon
        minx, miny, maxx, maxy = polygon.bounds
        bbox = [minx, miny, maxx, maxy]

        return bbox

    # Initialize variables
    max_span_overall = 0
    biggest_rest_station = None
    biggest_rest_station_bbox = None
    biggest_rest_station_name = None

    # Loop over all rows in the df, calculate the bbox and find the largest span
    for index, row in geodataframe.iterrows():
        bbox = polygon_to_bbox(row["geometry"])
        minx, miny, maxx, maxy = bbox
        x_span = maxx - minx
        y_span = maxy - miny

        max_span = max(x_span, y_span)

        if max_span > max_span_overall:
            max_span_overall = max_span
            biggest_rest_station = row["id_rest"]
            biggest_rest_station_bbox = [minx, miny, maxx, maxy]
            biggest_rest_station_name = row["name"]

    print(f"The biggest rest station is {biggest_rest_station_name} (id: {biggest_rest_station}) with a maximum bounding box span of {max_span_overall}")

    return biggest_rest_station

In [18]:
def calculate_bbox_dimensions(polygon):
    """
    Calculates the dimensions of the bounding box of a polygon in degrees.

    Parameters:
    polygon (Polygon): input polygon

    Returns:
    Width and height of the bounding box in degrees.
    """
    # Get the bounds of the polygon
    minx, miny, maxx, maxy = polygon.bounds
    # Calculate width and height in degrees
    width = maxx - minx
    height = maxy - miny
    return width, height

In [19]:
rest_station_id = find_biggest_rest_station(rest_stations_ver)

The biggest rest station is Neu-Nordsee (id: lon_9.9181171_lat_54.315654) with a maximum bounding box span of 0.008475900000000536


In [20]:
rest_station_id_height = rest_stations_ver[rest_stations_ver["id_rest"] == "lon_7.0388979_lat_52.3124962"]["geometry"].values[0]
print(rest_station_id_height)

POLYGON ((7.0388979 52.3124962, 7.0402467 52.3117209, 7.0470134 52.313444, 7.0462468 52.3142967, 7.043728 52.313585, 7.0435905 52.3134762, 7.0388979 52.3124962))


The conversion from coordinate distance to a distance in meters is taken from this [stackexchange](https://gis.stackexchange.com/questions/403637/convert-distance-in-shapely-to-kilometres) post.

In [21]:
def find_needed_image_pixel_size(polygon, WMS_resolution = 0.2):
    minx, miny, maxx, maxy = polygon.bounds

    print("The bounding box has the following coordinates", [minx, miny, maxx, maxy])

    x_line = shapely.LineString([(minx, miny), (maxx, miny)])
    y_line = shapely.LineString([(minx, miny), (minx, maxy)])

    geod = Geod(ellps="WGS84")

    length_x = math.ceil(geod.geometry_length(x_line))
    length_y = math.ceil(geod.geometry_length(y_line))

    print(f"The bounding box has a width of: {length_x}m and a height of: {length_y}m")

    max_length = max(length_x, length_y)

    needed_resolution = math.ceil(max_length / WMS_resolution)

    print("The needed minimum pixel size is:", needed_resolution, "pixels.")

In [22]:
find_needed_image_pixel_size(rest_station_id_height)

The bounding box has the following coordinates [7.0388979, 52.3117209, 7.0470134, 52.3142967]
The bounding box has a width of: 554m and a height of: 287m
The needed minimum pixel size is: 2770 pixels.


**Findings**

*   A pixel size of more than 2770 pixels guarantees that every image of a service stop has the highest resolution possible.
*   Plan: Download images of size 600m x 600m with a pixel size of 3000





## Determine download bounding boxes


### Define function

In [23]:
def create_image_bounding_box(polygon, bbox_size_m = 500):
    """
    Create an output bounding box that fully covers the input bounding box and is of size bbox_size_m x bbox_size_m.

    Parameters:
    polygon (shapely Polygon): input polygon
    bbox_size_m (int): size of the output bounding box

    Returns:
    The output bounding box as [left, bottom, right, top]
    """

    minx, miny, maxx, maxy = polygon.bounds

    # Calculate the centre of the input bounding box
    centre_x = (miny + maxy) / 2
    centre_y = (minx + maxx) / 2

    distance_from_centre = bbox_size_m / 2

    # Calculate the coordinates distance_from_centre meters north, south, east, and west of the center
    north = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=0).latitude
    south = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=180).latitude
    east = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=90).longitude
    west = distance(meters=distance_from_centre).destination((centre_x, centre_y), bearing=270).longitude

    # Calculate the maximum allowable shift in each direction to keep the input bbox within the output bbox
    max_shift_north = north - maxy
    max_shift_south = miny - south
    max_shift_east = east - maxx
    max_shift_west = minx - west

    # Randomly shift the center within the allowable range
    shift_x = random.uniform(-min(max_shift_west, distance_from_centre), min(max_shift_east, distance_from_centre))
    shift_y = random.uniform(-min(max_shift_south, distance_from_centre), min(max_shift_north, distance_from_centre))

    # Calculate the final output bounding box
    output_bbox = [
        west + shift_x,  # left
        south + shift_y, # bottom
        east + shift_x,  # right
        north + shift_y  # top
    ]

    return output_bbox

In [24]:
def determine_bbox_max_span_m(bbox):
    """
    Determine the size of a bounding box in meters.

    Parameters:
    bbox (list): bounding box as [left, bottom, right, top]
    """

    minx, miny, maxx, maxy = bbox

    x_line = shapely.LineString([(minx, miny), (maxx, miny)])
    y_line = shapely.LineString([(minx, miny), (minx, maxy)])

    geod = Geod(ellps="WGS84")

    length_x = math.ceil(geod.geometry_length(x_line))
    length_y = math.ceil(geod.geometry_length(y_line))

    print(f"The bounding box has a width of: {length_x}m and a height of: {length_y}m")

### Test function

In [25]:
# Extract random rest stop polygon

random_index = random.randint(0, 266)
print("Random index:", random_index)

# Extract polygon
test_polygon = rest_stations_ver.iloc[random_index]["geometry"]
print(test_polygon)

Random index: 60
POLYGON ((10.9810513 54.3627005, 10.9838103 54.3624764, 10.9834334 54.3628171, 10.9831467 54.3629105, 10.9826087 54.3630357, 10.9817587 54.3630758, 10.9813606 54.363019, 10.9811064 54.3629588, 10.9810513 54.3627005))


In [26]:
# Create bounding box
covering_test_bbox = create_image_bounding_box(test_polygon)
covering_test_bbox

[10.978111002250104,
 54.362123409211065,
 10.985802801113307,
 54.366615298013244]

In [27]:
# Check bounding box size in meter
determine_bbox_max_span_m(covering_test_bbox)

The bounding box has a width of: 501m and a height of: 501m


### Calculate bbox for every rest stop

In [28]:
# Set random seed
random.seed(103)

# Create image bounding boxes for every rest station
rest_stations_ver["bbox"] = rest_stations_ver['geometry'].apply(create_image_bounding_box)

In [29]:
# Change order of columns to comply with geopandas convention

rest_stations_ver = rest_stations_ver[["id_rest", "id_OSM_rest", "name", "bbox", "geometry"]]

In [30]:
rest_stations_ver.head()

,id_rest,id_OSM_rest,name,bbox,geometry
0,lon_8.6630029_lat_50.2542348,way/22568867,Schäferborn,"[8.662897964530508, 50.24947031826926, 8.66990...","POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
1,lon_11.3531491_lat_50.9378362,way/27549694,Habichtsfang,"[11.352126356256656, 50.93673132065365, 11.359...","POLYGON ((11.35315 50.93784, 11.35315 50.93783..."
2,lon_10.2712414_lat_53.331074,way/29376062,Roddau,"[10.264345359109937, 53.328233742563924, 10.27...","POLYGON ((10.27124 53.33107, 10.27096 53.33121..."
3,lon_9.6300119_lat_53.8303266,way/31128689,Steinburg,"[9.629200346595411, 53.82616873812249, 9.63679...","POLYGON ((9.63001 53.83033, 9.63273 53.82905, ..."
4,lon_9.6297338_lat_53.8301666,way/31128716,Steinburg,"[9.628319773449437, 53.82730601524238, 9.63591...","POLYGON ((9.62973 53.83017, 9.62965 53.83010, ..."


In [147]:
# Delete geometry column from rest_stations_ver
rest_stations_ver = rest_stations_ver.drop("geometry", axis = 1)

# Download rest_station_ver as csv
output_path_rest_stations_ver_bbox = "02_data_acquisition/verified_parking_data/rest_stations_ver_bbox_512_incl_quick.csv"
rest_stations_ver.to_csv(output_path_rest_stations_ver_bbox, index = False)

# Set up tile services

## WMS and layer information

This code was partly taken from: [OWSLib 0.29.3 documentation -> Examples -> Interact with a WMS](https://owslib.readthedocs.io/en/stable/notebooks/wms.html)

In [27]:
# Define WMS service and layer to be tested

test_wms_url = "https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS"
test_layer = "WMS_NI_DOP"

In [28]:
# Functions to return WMS/ layer information

def get_wms_information(wms_url):
    """
    Output all necessary WMS information for the WMS service provided.

    Parameters:
    wms_url: The GetCapabilities url of the WMS service

    Returns:
    A dictionary with version, url, title, abstract, provider_name, layer_names, operations, operations_urls, format_options
    """

    wms = WebMapService(wms_url, version="1.3.0")

    version = wms.identification.version
    title = wms.identification.title
    abstract = wms.identification.abstract
    provider_name = wms.provider.name
    layer_names = list(wms.contents)
    operations = [op.name for op in wms.operations]
    operations_urls = wms.getOperationByName("GetMap").methods
    format_options = wms.getOperationByName("GetMap").formatOptions

    wms_information = {"version": version,
                       "url": wms_url,
                       "title": title,
                       "abstract": abstract,
                       "provider_name": provider_name,
                       "layer_names": layer_names,
                       "operations": operations,
                       "operations_urls": operations_urls,
                       "format_options": format_options
                       }

    return wms_information

def get_layer_information(wms_url, layer_name):
    """
    Retrieve information about extent and crs_options of selected layer.

    Paramters:
    wms_url: The GetCapabilities url of the WMS service
    layer_name: The name of the selected layer

    Returns:
    A dictionary with layer_name, extent, crs_options.
    """

    wms = WebMapService(wms_url, version="1.3.0")

    extent = wms.contents[layer_name].boundingBoxWGS84
    crs_options = wms[layer_name].crsOptions

    layer_information = {"layer_name": layer_name,
                         "extent": extent,
                         "crs_options": crs_options
                         }

    return layer_information

In [29]:
# Return all relevant WMS service information
get_wms_information(test_wms_url)

{'version': '1.3.0',
 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS',
 'title': 'WMS NI DOP',
 'abstract': 'Digitale Orthophotos Niedersachsen, Bodenauflösung 20 cm (DOP20)',
 'provider_name': 'Landesamt für Geoinformation und Landesvermessung Niedersachsen (LGLN) - Landesbetrieb Landesvermessung und Geobasisinformation',
 'layer_names': ['WMS_NI_DOP', 'dop20'],
 'operations': ['GetCapabilities', 'GetMap', 'GetFeatureInfo'],
 'operations_urls': [{'type': 'Get',
   'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'},
  {'type': 'Post',
   'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'}],
 'format_options': ['image/png',
  'image/jpeg',
  'image/png; mode=8bit',
  'image/vnd.jpeg-png',
  'image/vnd.jpeg-png8',
  'application/x-pdf',
  'image/svg+xml',
  'image/tiff',
  'application/vnd.google-earth.kml+xml',
  'application/vnd.google-earth.kmz',
  'application/vn

In [30]:
# Return all relevant layer information
get_layer_information(test_wms_url, test_layer)

{'layer_name': 'WMS_NI_DOP',
 'extent': (6.505772, 51.153098, 11.754046, 54.148101),
 'crs_options': ['EPSG:31466',
  'EPSG:4258',
  'EPSG:31469',
  'EPSG:3035',
  'EPSG:25833',
  'EPSG:32632',
  'EPSG:25832',
  'EPSG:3857',
  'EPSG:31467',
  'EPSG:3034',
  'EPSG:32633',
  'EPSG:4647',
  'EPSG:4326',
  'EPSG:3044',
  'EPSG:3045',
  'EPSG:31468']}

## Return image from WMS bbox

In [31]:
# Define WMS service and layer for each German state

wms_urls = {
        'WMS_NI_DOP': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS', # Lower Saxony
        'bebb_dop20c': 'https://isk.geobasis-bb.de/mapproxy/dop20c/service/wms?Request=GetCapabilities&Service=WMS', # Brandenburg & Berlin
        'th_dop': 'https://www.geoproxy.geoportal-th.de/geoproxy/services/DOP?REQUEST=GetCapabilities&version=1.1.1&service=WMS', # Thuringia
        'lsa_lvermgeo_dop20_2': 'https://www.geodatenportal.sachsen-anhalt.de/wss/service/ST_LVermGeo_DOP_WMS_OpenData/guest?', # Saxony-Anhalt
        'he_dop_rgb': 'https://gds-srv.hessen.de/cgi-bin/lika-services/ogc-free-images.ows?', # Hessia
        'sh_dop20_rgb': 'https://dienste.gdi-sh.de/WMS_SH_DOP20col_OpenGBD?Service=wms&version=1.3.0&request=getCapabilities', # Schleswig-Holstein
    }

*   sh_dop20_rgb has an overly large, inaccurate extent. For this reason, this layer is tested last

In [32]:
# Print WMS and layer information for all selected Länder layers
counter = 1

for layer, url in wms_urls.items():
    print(f"wms_{counter}", get_wms_information(url))
    print(f"layer_{counter}", get_layer_information(url, layer))
    print("\n")
    counter += 1

wms_1 {'version': '1.3.0', 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS', 'title': 'WMS NI DOP', 'abstract': 'Digitale Orthophotos Niedersachsen, Bodenauflösung 20 cm (DOP20)', 'provider_name': 'Landesamt für Geoinformation und Landesvermessung Niedersachsen (LGLN) - Landesbetrieb Landesvermessung und Geobasisinformation', 'layer_names': ['WMS_NI_DOP', 'dop20'], 'operations': ['GetCapabilities', 'GetMap', 'GetFeatureInfo'], 'operations_urls': [{'type': 'Get', 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'}, {'type': 'Post', 'url': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?'}], 'format_options': ['image/png', 'image/jpeg', 'image/png; mode=8bit', 'image/vnd.jpeg-png', 'image/vnd.jpeg-png8', 'application/x-pdf', 'image/svg+xml', 'image/tiff', 'application/vnd.google-earth.kml+xml', 'application/vnd.google-earth.kmz', 'application/vnd.mapbox-vector-tile', 'applic

**Findings:**

*   All layers have the option to select the EPSG:4326 coordinate system, a coordinate system on the WGS84 reference ellipsoid
*   WGS84 is also used as the standard coordinates system used by GSP and also OSM
*   Only Niedersachsen, Thüringen and Hessen can export in Tiff format

In [33]:
def retrieve_layer_extents(wms_urls):
        """
        Retrieve layer extents of the WMS urls.

        Parameters:
        wms_urls (dict): dictionary of layer names and WMS GetCapability URLs

        Returns:
        A dictionary of the extents of the WMS layers.
        """
        wms_layer_extents = {}

        for layer_name, url in wms_urls.items():
            bounding_box = get_layer_information(url, layer_name)["extent"]
            wms_layer_extents[layer_name] = bounding_box

        return wms_layer_extents

In [34]:
# Extract layer extents
layer_extents = retrieve_layer_extents(wms_urls)
print(layer_extents)

{'WMS_NI_DOP': (6.505772, 51.153098, 11.754046, 54.148101), 'bebb_dop20c': (11.152768795679583, 51.2635170116316, 15.009068839315324, 53.61004915755329), 'th_dop': (9.70043908, 50.13516795, 12.75958006, 51.7188448), 'lsa_lvermgeo_dop20_2': (10.5092, 50.8927, 13.3233, 53.0769), 'he_dop_rgb': (7.41867, 49.25, 10.5, 51.7596), 'sh_dop20_rgb': (0.105946742406, 45.237542736, 20.4488912945, 56.8478734515)}


In [35]:
def create_png_from_bbox(bbox, wms_urls, layer_extents, size = 2000):
    """
    Create square png map from a bounding box sourced from WMS services.

    Parameters:
    bbox (tuple): (left, bottom, right, top)
    wms_urls (dict): dictionary containing layer_names as keys and urls as values
    layer_extents (dict): dictionary containing layer_names as keys and layer extent tuples as values
    size (int): determines the size of the output png in pixels

    Returns:
    A square png image of the map.
    """


    def bbox_intersects(bbox1, bbox2):
        """
        Check if two bounding boxes intersect.

        Parameters:
        bbox1, bbox2 (tuple): tuple of bbox corners (left, bottom, right, top)

        Returns:
        True if bboxes intersect, False otherwise
        """
        return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or
                    bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])


    def is_blank_image(image):
        """
        Check whether an image is blank or not.

        Parameters:
        image (image object): image of WMS service

        Returns:
        True if image is blank, False otherwise.
        """
        try:
            # Open the image from the image object
            image = Image.open(io.BytesIO(image.read()))

            # Convert the image to a numpy array
            image_array = np.array(image)

            # Depending on the image mode, check whether image is completely white or black
            if image.mode == 'RGB':
                # Check if the image is completely white or black
                is_white = np.all(image_array == [255, 255, 255], axis=(-1)).all()
                is_black = np.all(image_array == [0, 0, 0], axis=(-1)).all()
            elif image.mode == 'RGBA':
                # Check if the image is completely white or black
                is_white = np.all(image_array[:, :, :3] == [255, 255, 255], axis=(-1)).all()
                is_black = np.all(image_array[:, :, :3] == [0, 0, 0], axis=(-1)).all()
            else:
                # Raise error for other image mode
                raise ValueError("Unsupported image mode.")

            return is_white or is_black

        except Exception as e:
            print(f"An error occurred: {e}")
            return False


    def determine_layer_for_bbox(bbox, wms_urls, layer_extents):
        """
        Determine which layer to use based for the provided bounding box.

        Parameters:
        bbox (tuple): bounding box (left, bottom, right, top)
        wms_urls (dict): dictionary containing layer_names as keys and urls as values
        layer_extents: dictionary of layer names and their extent bounding boxes

        Returns:
        Layer name or None if no intersection
        """
        # Check WMSs one after the other until image is returned
        for layer, extent in layer_extents.items():
            wms = WebMapService(wms_urls[layer])
            response = wms.getmap(layers=[layer],
                              srs='EPSG:4326',
                              bbox=bbox,
                              size=(2, 2),
                              format='image/png')
            # First, check if bounding box intersects
            if not bbox_intersects(bbox, extent):
                print(f"{layer} does not contain bounding box.")
            # Second, check whether image is blank
            elif is_blank_image(response):
                print(f"{layer} returns blank image.")
            # Return image
            else:
                print(f"{layer} returns image.")

                return layer
        return None

    # Determine the layer that intersects the bounding box
    chosen_layer = determine_layer_for_bbox(bbox, wms_urls, layer_extents)

    if chosen_layer == None:
        raise ValueError("No layer intersects with the provided bounding box.")
    else:
        # Correctly create a WebMapService object from chosen layer
        wms = WebMapService(wms_urls[chosen_layer])
        response = wms.getmap(layers=[chosen_layer],
                              srs='EPSG:4326',
                              bbox=bbox,
                              size=(size, size),
                              format='image/png')
    # Return the image
    return response

In [36]:
def pixel_dimensions(image):
    """
    Return pixel dimensions of an image.

    Parameters:
    image (PIL Image object)

    Returns:
    A tuple of width and height dimensions of the input image.
    """

    image_data = BytesIO(image.read())
    image_loaded = Image.open(image_data)

    # Get image dimensions (Width x Height)
    width, height = image_loaded.size

    return (width, height)

# Download images

## Define downloading functions

In [37]:
def download_as_png(image, filepath):
    """
    Downloads
    """
    # Assuming img_new is your image response object
    image_data = BytesIO(image.read())

    # Write the image data to a file
    with open(filepath, 'wb') as file:
        file.write(image_data.getvalue())

    # Provide the file path for downloading
    #print("Downloaded to", filepath)

In [38]:
def download_tif_from_png(png_path, tif_path, bbox):

    # Load the PNG image as a Rasterio dataset
    png = rasterio.open(png_path)

    # Read the data from the dataset into a NumPy array
    image_array = png.read()

    # Define transformation from bounding box
    left, bottom, right, top = bbox
    transform = from_bounds(left, bottom, right, top, png.width, png.height)

    # Update metadata
    meta = {
        'driver': 'GTiff',
        'dtype': png.dtypes[0],
        'nodata': None,
        'width': png.width,
        'height': png.height,
        'count': png.count,
        'crs': CRS.from_epsg(4326),  # WGS84
        'transform': transform
    }

    # Write the array to a new TIFF file
    with rasterio.open(tif_path, 'w', **meta) as dst:
        dst.write(image_array)

In [39]:
def delete_file(file_path):
    # Check if the file exists
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File '{file_path}' does not exist.")

## Download tifs

In [40]:
def mass_download_tif(geodataframe, wms_urls, layer_extents, size = 2560):

    for index, row in geodataframe.iterrows():
        bbox = row["bbox"]
        id_rest = row["id_rest"]
        name = row["name"]

        print(f"Downloading image {index}: {name} ({id_rest})\n")

        # Create map
        response = create_png_from_bbox(bbox, wms_urls, layer_extents, size)

        # Define base directory
        base_dir = "02_data_acquisition/tif_download_512_incl_quick"
        
        # Ensure the directory exists
        os.makedirs(base_dir, exist_ok=True)

        # Save map as png
        #file_path_png = f"/content/drive/MyDrive/Master Thesis/01 Data Acquisition/tif_download/{id_rest}_{name}.png"
        file_path_png = f"02_data_acquisition/tif_download_512_incl_quick/{id_rest}_{name}.png"
        download_as_png(response, file_path_png)

        # Safe tif from png
        file_path_tif = f"02_data_acquisition/tif_download_512_incl_quick/{id_rest}_{name}.tif"
        download_tif_from_png(file_path_png, file_path_tif, bbox)

        # Delete png
        delete_file(file_path_png)

        print(f"\nDownload completed for image {index}: {name} ({id_rest})")
        print("----------------------------------------------------------------------------------------- \n")

In [46]:
rest_stations_ver_rest = rest_stations_ver[233:]

In [47]:
rest_stations_ver_rest

,id_rest,id_OSM_rest,name,bbox,geometry
233,lon_12.5320732_lat_53.0701823,way/376036851,Autohof Herzsprung,"[12.529794017739789, 53.06627620962568, 12.537...","POLYGON ((12.53207 53.07018, 12.53147 53.07008..."
234,lon_9.9208466_lat_53.8213382,way/376516530,Moorkaten-West,"[9.916739806645424, 53.81879345032405, 9.92433...","POLYGON ((9.92085 53.82134, 9.92104 53.82124, ..."
235,lon_12.4887602_lat_51.9716412,way/376544328,Rosselquelle,"[12.482407729179878, 51.97044805887731, 12.489...","POLYGON ((12.48876 51.97164, 12.48867 51.97180..."
236,lon_10.891129_lat_54.2412856,way/376558935,Damlos,"[10.890390567388298, 54.23694523265666, 10.898...","POLYGON ((10.89113 54.24129, 10.89100 54.24124..."
237,lon_10.856376_lat_54.1551497,way/376559800,Hasselburger Mühle,"[10.852610349904223, 54.151964830272135, 10.86...","POLYGON ((10.85638 54.15515, 10.85621 54.15517..."
...,...,...,...,...,...
291,lon_12.7580653_lat_52.8778128,way/915630046,Am Rhinluch,"[12.754311642753441, 52.87373430027324, 12.761...","POLYGON ((12.75807 52.87781, 12.75786 52.87778..."
292,lon_7.2005154_lat_52.2816225,way/988014767,Gut Friedrichstal,"[7.196198912998569, 52.27916482977726, 7.20352...","POLYGON ((7.20052 52.28162, 7.20053 52.28167, ..."
293,lon_8.5597202_lat_49.5969765,way/992004892,Wildbahn,"[8.554638173536537, 49.59453943341597, 8.56155...","POLYGON ((8.55972 49.59698, 8.55958 49.59693, ..."
294,lon_10.7306355_lat_50.559048,way/1171916133,Adlersberg,"[10.724665021750997, 50.55768307908295, 10.731...","POLYGON ((10.73064 50.55905, 10.73067 50.55851..."


In [48]:
mass_download_tif(rest_stations_ver_rest, wms_urls, layer_extents, size = 2560)


WMS_NI_DOP does not contain bounding box.
bebb_dop20c returns image.


/Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas/.venv/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)



Download completed for image 233: Autohof Herzsprung (lon_12.5320732_lat_53.0701823)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP returns blank image.
bebb_dop20c does not contain bounding box.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 does not contain bounding box.
he_dop_rgb does not contain bounding box.
sh_dop20_rgb returns image.

Download completed for image 234: Moorkaten-West (lon_9.9208466_lat_53.8213382)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP does not contain bounding box.
bebb_dop20c returns blank image.
th_dop does not contain bounding box.
lsa_lvermgeo_dop20_2 returns image.

Download completed for image 235: Rosselquelle (lon_12.4887602_lat_51.9716412)
----------------------------------------------------------------------------------------- 


WMS_NI_DOP does not contain bounding box.
bebb_dop20c does not contain bounding box.
th_do

Kapellenberg (index 232) does not return an image (with quick parking).

## Test Download

In [97]:
# Define test index
test_index = 4

test_bbox = rest_stations_ver.iloc[test_index].loc["bbox"]
test_id_rest = rest_stations_ver.iloc[test_index].loc["id_rest"]
test_name = rest_stations_ver.iloc[test_index].loc["name"]

print("Test bounding box:", test_bbox)
print("Test id box:", test_id_rest)
print("Test service station:", test_name)

Test bounding box: [11.006819328464454, 50.83506964851384, 11.013917149461077, 50.83956422673636]
Test id box: lon_11.0102562_lat_50.8353109
Test service station: Dornheimer Rieth


In [98]:
# Create map for test service station
response = create_png_from_bbox(test_bbox, wms_urls, layer_extents, size = 2560)

WMS_NI_DOP does not contain bounding box.
bebb_dop20c does not contain bounding box.
th_dop returns image.


In [ ]:
# Display service station (usually commented to safe memory)
displayImage(response.read())

In [76]:
# Print type and pixel dimensions
print(type(response))
print("Image pixel dimensions", pixel_dimensions(response))

<class 'owslib.util.ResponseWrapper'>
Image pixel dimensions (2560, 2560)


In [60]:
# Download image as PNG

file_path_png = f"02_data_acquisition/test_image_download/{test_id_rest}_{test_name}.png"
download_as_png(response, file_path_png)

In [62]:
# Download image as tif from png

file_path_tif = f"02_data_acquisition/test_image_download/{test_id_rest}_{test_name}.tif"
download_tif_from_png(file_path_png, file_path_tif, test_bbox)

/Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas/.venv/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [63]:
# Delete PNG function

def delete_file(file_path):
    # Check if the file exists
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File '{file_path}' has been deleted.")
    else:
        print(f"File '{file_path}' does not exist.")

In [64]:
# Delete Png
file_path_png = file_path_png
delete_file(file_path_png)

File '02_data_acquisition/test_image_download/lon_12.1106921_lat_51.4564157_Kabelsketal.png' has been deleted.


# Delete service stations from gpd with faulty images

In [49]:
# Load the GeoJSON files of verified rest stops
parking_areas_ver = gpd.read_file("02_data_acquisition/verified_parking_data/parking_areas_ver_incl_quick.geojson") # polygons of verified car and truck parking space
rest_stations_ver = gpd.read_file("02_data_acquisition/verified_parking_data/rest_stations_ver_incl_quick.geojson") # polygons of verified rest stops

# Load bbox csv
rest_stations_ver_bbox = pd.read_csv("02_data_acquisition/verified_parking_data/rest_stations_ver_bbox_512_incl_quick.csv")

In [50]:
# Check shapes of dataframes

print("Shape of parking_areas_ver before dropping:", parking_areas_ver.shape)
print("Shape of rest_stations_ver before dropping:", rest_stations_ver.shape)
print("Shape of rest_stations_ver_bbox before dropping:", rest_stations_ver_bbox.shape, "\n")

# Drop rows referring to Kapellenberg (lon_12.2060105_lat_51.4912736) and Brockenblick (lon_10.6579729_lat_51.9212477) by id_rest
rest_stations_ver = rest_stations_ver[~rest_stations_ver["id_rest"].isin(["lon_12.2060105_lat_51.4912736", "lon_10.6579729_lat_51.9212477"])]
rest_stations_ver_bbox = rest_stations_ver_bbox[~rest_stations_ver_bbox["id_rest"].isin(["lon_12.2060105_lat_51.4912736", "lon_10.6579729_lat_51.9212477"])]
parking_areas_ver = parking_areas_ver[~parking_areas_ver["id_rest"].isin(["lon_12.2060105_lat_51.4912736", "lon_10.6579729_lat_51.9212477"])]

print("Shape of parking_areas_ver after dropping:", parking_areas_ver.shape)
print("Shape of rest_stations_ver after dropping:", rest_stations_ver.shape)
print("Shape of rest_stations_ver_bbox after dropping:", rest_stations_ver_bbox.shape)


Shape of parking_areas_ver before dropping: (1304, 4)
Shape of rest_stations_ver before dropping: (296, 4)
Shape of rest_stations_ver_bbox before dropping: (296, 4) 

Shape of parking_areas_ver after dropping: (1300, 4)
Shape of rest_stations_ver after dropping: (295, 4)
Shape of rest_stations_ver_bbox after dropping: (295, 4)


In [165]:
# Download parking_areas_ver, rest_stations_ver as GeoJSN and rest_stations_ver_bbox as csv

# Specify the path where you want to save the GeoJSON file
output_path_rest_stations_ver_final = "02_data_acquisition/verified_parking_data/rest_stations_ver_incl_quick_final.geojson"
output_path_parking_areas_ver_final = "02_data_acquisition/verified_parking_data/parking_areas_ver_incl_quick_final.geojson"
output_path_rest_stations_ver_bbox_512_final = "02_data_acquisition/verified_parking_data/rest_stations_ver_bbox_512_incl_quick_final.csv"

# Export rest_stations_ver_final and parking_areas_ver_final as GeoJSON
rest_stations_ver.to_file(output_path_rest_stations_ver_final, driver='GeoJSON')
parking_areas_ver.to_file(output_path_parking_areas_ver_final, driver='GeoJSON')

# Export rest_stations_ver_bbox_final to CSV
rest_stations_ver_bbox.to_csv(output_path_rest_stations_ver_bbox_512_final, index = False)